## Setup : chargement des données et des modèles

In [1]:
import os


# Dataset batch size
BATCH_SIZE = 256
# Directory containing the dataset pickles
DATA_DIR = os.path.join("data", "pickle_img_datasets")
# Directory containing images
IMAGES_DIR = os.path.join("data", "images", "image_train")

In [2]:
from tensorflow import keras
import tensorflow as tf
import os
import pickle
from src.data import data


# Load data
X_train = pickle.load(
    open(os.path.join(DATA_DIR, "X_train.pkl"), "rb")).fillna("")
X_test = pickle.load(
    open(os.path.join(DATA_DIR, "X_test.pkl"), "rb")).fillna("")
y_test = pickle.load(open(os.path.join(DATA_DIR, "y_test.pkl"), "rb"))

# Extract the features to be ready for preprocessing
X_train_features = X_train['designation'] + " " + X_train['description']
X_test_features = X_test['designation'] + " " + X_test['description']

# Store the file path to images in variables
X_train_images = data.get_imgs_filenames(
    X_train["productid"], X_train["imageid"], IMAGES_DIR)
X_test_images = data.get_imgs_filenames(
    X_test["productid"], X_test["imageid"], IMAGES_DIR)

# Define DataFrame names for preprocessing
X_train_features.name = "X_train"
X_test_features.name = "X_test"

# Load text model
text_model = keras.models.load_model(
    os.path.join("data", "models", "mlp_model_v2.h5"))

# Load image model
image_model = keras.models.load_model(
    os.path.join("data", "models", "cnn_mobilenetv2_keras",
                 "cnn_mobilenetv2.h5"),
    compile=False)

2023-04-24 18:07:38.980662: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 18:07:39.001381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 18:07:39.336147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-24 18:07:40.077569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Preprocessing du texte

In [3]:
from src.data.text_preproc_pipeline import TextPreprocess
from src.data.vectorization_pipeline import TfidfStemming


# Data preprocessing
text_preprocessor = TextPreprocess(TfidfStemming())
text_preprocessor.fit(X_train_features)
X_test_preproc = text_preprocessor.transform(X_test_features)

INFO:textpipeline:class:TfidfStemming
INFO:textpipeline:TextPreprocess.fit X_train 39.08 seconds
INFO:textpipeline:TextPreprocess.transform X_test 9.77 seconds


# Prédictions du modèle texte

In [4]:
from sklearn.metrics import accuracy_score
from src.data.data import convert_sparse_matrix_to_sparse_tensor, get_model_prediction


# Predict the categories of X_test
y_pred_text = text_model.predict(
    convert_sparse_matrix_to_sparse_tensor(X_test_preproc))

# Display the accuracy score
print("Text model accuracy score:", accuracy_score(
    y_test, get_model_prediction(y_pred_text)))

2023-04-24 18:08:30.860401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [16984,3]
	 [[{{node Placeholder/_0}}]]


531/531 [==============================] - 1s 2ms/step
Text model accuracy score: 0.02773198304286387


# Preprocessing des images

In [5]:
from src.data.data import PRDTYPECODE_DIC, to_simplified_prdtypecode


def open_resize_img(filename: str, y) -> None:
    """
    Open image using the filename and return a resized version of it ready for the image model.

    Argument:
    - filename: complete path to image file including the extension.

    Return:
    - Image matrix in a tensor.
    """
    img = tf.io.read_file(filename)
    img = tf.io.decode_jpeg(img, channels=3)
    return (tf.image.resize(img, [224, 224]), y)


# Convert the prdtypecode to their equivalent in a range from 0 to 26
y_test_simplified = to_simplified_prdtypecode(y_test)
# Transforms y_test to a one hot version
y_test_categorical = tf.keras.utils.to_categorical(
    y_test_simplified, num_classes=len(PRDTYPECODE_DIC.keys()))

# Create the Dataset to feed the model with correctly sized images by batch
test_images_dataset = tf.data.Dataset.from_tensor_slices((X_test_images, y_test_categorical)) \
    .map(open_resize_img) \
    .batch(BATCH_SIZE)

# Prédictions du modèle image

In [6]:
from sklearn.metrics import accuracy_score
from src.data.data import get_model_prediction


# Predict the categories of X_test
y_pred_image = image_model.predict(test_images_dataset)

# Display the accuracy score
print("Image model accuracy score:", accuracy_score(
    y_test, get_model_prediction(y_pred_image)))

2023-04-24 18:08:32.434674: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [16984,27]
	 [[{{node Placeholder/_1}}]]
2023-04-24 18:08:33.508247: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-04-24 18:08:33.998170: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


67/67 [==============================] - 24s 304ms/step
Image model accuracy score: 0.5444536033914272
